In [1]:
## Loading the required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data to work with

In [ ]:
## Create the datasets using significant features
from scipy.stats import ttest_ind
from collections import defaultdict

## high utilization and non-fraudulent
significant = defaultdict(int)
for dataset in data_merged:
    pvalues = {}
    for f in dataset.columns:
        group1 = dataset.loc[flag == 'high utilization', f]
        group2 = dataset.loc[flag == 'non-fraudulent', f]
        val = ttest_ind(group1.values, group2.values)[1]
        if str(val) != 'nan' :
            pvalues[f] = val
            
    p = 0.1/len(dataset.columns)

    for f in pvalues:
        if pvalues[f] < p:
            significant[f] += 1
            
significant_codes_HN = list(set([i[:5] for i in significant.keys()]))

## fraudulent and non-fraudulent
significant = defaultdict(int)
for dataset in data_merged:
    pvalues = {}
    for f in dataset.columns:
        group1 = dataset.loc[flag == 'fraudulent', f]
        group2 = dataset.loc[flag == 'non-fraudulent', f]
        val = ttest_ind(group1.values, group2.values)[1]
        if str(val) != 'nan' :
            pvalues[f] = val
            
    p = 0.1/len(dataset.columns)

    for f in pvalues:
        if pvalues[f] < p:
            significant[f] += 1
            
significant_codes_FN = list(set([i[:5] for i in significant.keys()]))

## fraudulent and high utilization
significant = defaultdict(int)
for dataset in data_merged:
    pvalues = {}
    for f in dataset.columns:
        group1 = dataset.loc[flag == 'fraudulent', f]
        group2 = dataset.loc[flag == 'high utilization', f]
        val = ttest_ind(group1.values, group2.values)[1]
        if str(val) != 'nan' :
            pvalues[f] = val
            
    p = 0.1/len(dataset.columns)

    for f in pvalues:
        if pvalues[f] < p:
            significant[f] += 1
            
significant_codes_FH = list(set([i[:5] for i in significant.keys()]))

## Feature selection
codes_to_use = list(set([*significant_codes_HN, *significant_codes_FN, *significant_codes_FH]))

data_merged_afs = []
for i in range(len(data_merged)):
    clmns = list(data_merged[i].columns)
    clmns_to_use = [hcpcs for hcpcs in clmns if hcpcs[0:5] in codes_to_use]
    data_merged_afs.append(data_merged[i][clmns_to_use])
        
# [x_MP, x_NoS, x_Ratio] = data_merged_afs

In [ ]:
## Create the datasets using only overused CPT codes

overused_codes_before = ['14061', '65800', '65855', '67973', '67975', '68326', '68335', 
                         '76510', '76511', '76512', '76513', '92060', '92100', '92235', 
                         '92240', '92275', '92284', '95004', '95930']

overused_codes_march8 = ['65100', '65210', '65222', '65400', '65426', '65722', '65815', 
                        '66030', '66175', '66183', '66761', '66982', '67938', '68761', 
                        '68801', '76514', '92020', '92025', '92065', '92083', '92140', 
                        '92225', '92226', '92250', '92286', '99354']

## Get rid of the 1-3 codes
# em_codes = ['99201', '99202', '99203', '99204', '99205',
#             '99211', '99212', '99213', '99214', '99215',
#             '99221', '99222', '99223',
#             '99231', '99232', '99233',
#             '99241', '99242', '99243', '99244', '99245',
#             '99251', '99252', '99253', '99254', '99255',
#             '99281', '99282', '99283', '99284', '99285',
#             '99304', '99305', '99306', '99307', '99308', '99309', '99310']

em_codes = ['99204', '99205',
            '99214', '99215',
            '99284', '99285',
            '99304', '99305', '99306', '99307', '99308', '99309', '99310']

# ## Get rid of 92002 and 92012
# eye_visit_codes = ['92002', '92004', 
#                    '92012', '92014']

eye_visit_codes = ['92004', 
                   '92014']

## Feature selection
codes_to_use = [*overused_codes_before, *overused_codes_march8, *em_codes, *eye_visit_codes]

data_merged_afs = []
for i in range(len(data_merged)):
    clmns = list(data_merged[i].columns)
    clmns_to_use = [hcpcs for hcpcs in clmns if hcpcs[0:5] in codes_to_use]
    data_merged_afs.append(data_merged[i][clmns_to_use])
        
[x_MP, x_NoS, x_Ratio] = data_merged_afs

**The following codes could not be found in our merged dataset:**

65100, 65722, 67973

Consultation Codes: 99241, 99242, 99243, 99244, 99245, 99251, 99252, 99253, 99254, 99255

In [ ]:
x_MP_train, x_MP_unlabeled = x_MP.loc[flag != 'unlabeled'], x_MP.loc[flag == 'unlabeled']
x_NoS_train, x_NoS_unlabeled = x_NoS.loc[flag != 'unlabeled'], x_NoS.loc[flag == 'unlabeled']
x_Ratio_train, x_Ratio_unlabeled = x_Ratio.loc[flag != 'unlabeled'], x_Ratio.loc[flag == 'unlabeled']

y_train = flag[flag != 'unlabeled']

## Choosing a dedicated validation set
idx_fraud = y_train[y_train == 'fraudulent'].index
idx_high_utilization = y_train[y_train == 'high utilization'].index
idx_non_fraud = y_train[y_train == 'non-fraudulent'].index


idx_fraud_val = np.sort(np.random.choice(idx_fraud, size=len(idx_fraud) // 4, replace=False))
idx_high_utilization_val = np.sort(np.random.choice(idx_high_utilization, size=len(idx_high_utilization) // 4, replace=False))
idx_non_fraud_val = np.sort(np.random.choice(idx_non_fraud, size=len(idx_non_fraud) // 4, replace=False))
idx_val = np.sort(np.concatenate((idx_fraud_val, idx_high_utilization_val, idx_non_fraud_val)))

x_MP_val = x_MP_train.loc[idx_val]
x_NoS_val = x_NoS_train.loc[idx_val]
x_Ratio_val = x_Ratio_train.loc[idx_val]
y_val = y_train[idx_val]
len_val = len(y_val)

x_MP_train = x_MP_train.drop(index=idx_val)
x_NoS_train = x_NoS_train.drop(index=idx_val)
x_Ratio_train = x_Ratio_train.drop(index=idx_val)
y_train = y_train.drop(index=idx_val)
len_train = len(y_train)

x_stacked = [np.vstack((x_MP_train, x_MP_val)), np.vstack((x_NoS_train, x_NoS_val)), np.vstack((x_Ratio_train, x_Ratio_val))]
y_stacked = np.concatenate((y_train, y_val))

x_concat = np.concatenate((x_stacked[0], x_stacked[1], x_stacked[2]), axis=1)

## Sparsity check

In [ ]:
## Using spy() method to see how sparse our input matrices are
# plt.figure(figsize=(10, 40))
# plt.spy(x_concat)

fig, axs = plt.subplots(1, 3, figsize=(20, 20))
titles = ['MP', 'NoS', 'Ratio']
for i in range(len(x_stacked)):
    axs[i].spy(x_stacked[i])
    axs[i].set_title(titles[i])
plt.tight_layout()

In [ ]:
## Convert dataset type from dense to sparse
# from scipy.sparse import csr_matrix

# x_concat_sparse = csr_matrix(x_concat)

## Implement Logistic Regression, SVM, and K-Nearest Neighbor classifiers

In [ ]:
## Load the required modules
from sklearn.model_selection import StratifiedKFold, cross_val_score, validation_curve, GridSearchCV, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, SVR, LinearSVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA, NMF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, RFE, chi2, VarianceThreshold, f_classif

In [ ]:
## Using a balanced dataset

# idx_fraud = y_train[y_train == 'fraudulent'].index
# idx_non_fraud = y_train[y_train == 'non-fraudulent'].index

# min_len = min(len(idx_fraud), len(idx_non_fraud))
# idx_fraud_b = np.sort(np.random.choice(idx_fraud, size=min_len, replace=False))
# idx_non_fraud_b = np.sort(np.random.choice(idx_non_fraud, size=min_len, replace=False))
# idx_to_keep = np.sort(np.concatenate((idx_fraud_b, idx_non_fraud_b)))

# x_MP_train_b = x_MP_train.loc[idx_to_keep]
# x_NoS_train_b = x_NoS_train.loc[idx_to_keep]
# x_Ratio_train_b = x_Ratio_train.loc[idx_to_keep]
# y_train_b = y_train[idx_to_keep]

In [ ]:
## Data standardization
x_SCALE = []
for x in x_stacked:
    x_SCALE.append(StandardScaler().fit_transform(x))
    
fig, axs = plt.subplots(1, 3, figsize=(20, 20))
label = ['MP', 'NoS', 'Ratio']
for i in range(len(x_SCALE)):
    axs[i].spy(x_SCALE[i])
    axs[i].set_title('{}'.format(label[i]))
plt.tight_layout()

In [ ]:
## Feature selection using statistical methods (Filter methods)
# x_STAT = [SelectKBest(chi2, k=50).fit_transform(x_concat, y_stacked)]
x_STAT = []
for x in x_stacked:
    x_STAT.append(VarianceThreshold().fit_transform(x, y_stacked))

# x_STAT_2 = []
# for x in x_STAT:
#     x_STAT_2.append(SelectKBest(f_classif, k=50).fit_transform(x, y_stacked))

fig, axs = plt.subplots(1, 3, figsize=(20, 20))
label = ['MP', 'NoS', 'Ratio']
for i in range(len(x_STAT)):
    axs[i].spy(x_STAT[i])
    axs[i].set_title('{}'.format(label[i]))
plt.tight_layout()

In [ ]:
## Dimensionality reduction using PCA 
labels = ['MP', 'NoS', 'Ratio']
for i in range(3):
    pca = PCA().fit(x_STAT[i])
    plt.plot(np.cumsum(pca.explained_variance_ratio_), label=labels[i])
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance')
    plt.legend()

n = 50 # number of components to keep
x_PCA = []
for x in x_STAT:
    pca = PCA(n_components=n)
    z = StandardScaler().fit_transform(x)
    pca.fit(z)
    x_PCA.append(pca.transform(z))

In [ ]:
## Dimensionality reduction using NMF
model = NMF(n_components=100, init='random', random_state=0, max_iter=10000)
x_NMF = []
for x in x_STAT:
    x_NMF.append(model.fit_transform(x))

In [ ]:
## Feature selection using RFE (Wrapper methods)
estimator = DecisionTreeClassifier(max_depth=100, max_leaf_nodes=100, min_samples_leaf=6, random_state=0)
# estimator =  LinearSVC(C=1, max_iter=1e6, penalty='l1', dual=False)
selector = RFE(estimator, n_features_to_select=50, step=1)

# z = StandardScaler().fit_transform(x_concat)
# selector.fit(z, y_stacked)
# x_RFE = [selector.fit_transform(z, y_stacked)]

x_RFE = []
for x in x_STAT:
    z = StandardScaler(with_mean=False).fit_transform(x)
    selector = selector.fit(z, y_stacked)
    x_RFE.append(selector.fit_transform(z, y_stacked))

In [ ]:
## Training and choosing the best model using the dedicated validation set or cross validation
# split = PredefinedSplit(test_fold=np.repeat([-1, 0], [len_train, len_val]))
split = StratifiedKFold(4, shuffle=True, random_state=0)

x_train = x_RFE

parameters = {'svc__C':np.logspace(-4,4,10)}
# parameters = {'lgr__C':np.logspace(-3,3,10)}
# parameters = {'dt__max_depth':range(1,5), 'dt__min_samples_leaf':range(1,8),'dt__min_samples_split':[2, 3, 4],'dt__max_leaf_nodes':range(2,8)}
# parameters = {'rf__max_depth':range(3,6),'rf__min_samples_leaf':range(2,6),'rf__min_samples_split':[2, 3, 4]}
# parameters = {'ab__n_estimators':range(1,20),'ab__learning_rate':np.linspace(0.01,1,10)}
# parameters = {'qda__reg_param':np.logspace(-4,-1,10)}

scores = []
hypers = []

for x in x_train:
#     pipelines might need ('scaler', StandardScaler())
    clf_pipe = Pipeline([('svc', SVC(max_iter=1e9, kernel='rbf', gamma='auto'))])#
#     clf_pipe = Pipeline([('scaler', StandardScaler()), ('svc', LinearSVC(max_iter=1e6, penalty='l2', dual=False))])
#     clf_pipe = Pipeline([('scaler', StandardScaler()), ('lgr', LogisticRegression(max_iter=1e6, solver='lbfgs', penalty='l2'))])
#     clf_pipe = Pipeline([('dt',DecisionTreeClassifier(random_state=0))])
#     clf_pipe = Pipeline([('rf',RandomForestClassifier(random_state=0))])
#     clf_pipe = Pipeline([('ab',AdaBoostClassifier(random_state=0))])
#     clf_pipe = Pipeline([('qda', QuadraticDiscriminantAnalysis())])
    clf_grid = GridSearchCV(clf_pipe, parameters, cv=split)
    clf_grid.fit(x, y_stacked) 
    clf_best = clf_grid.best_estimator_
    scores.append(np.mean(cross_val_score(clf_best, x, y_stacked, cv=split)))
    hypers.append(clf_best.named_steps['svc'])
    
print('Best scores:', scores)
print('Best hyperparameters:', hypers)

In [ ]:
## Two-stage SVM

    
x_train = [x_stacked_SVM]

ps = PredefinedSplit(test_fold=np.repeat([-1, 0], [len_train, len_val]))
       
parameters = {'svc__C':np.logspace(-4,4,10)}    

scores = []
hypers = []

for x in x_train:
    # ('scaler', StandardScaler()), 
    clf_pipe = Pipeline([('svc', LinearSVC(max_iter=1e6, penalty='l2', dual=False))])
    clf_grid = GridSearchCV(clf_pipe, parameters, cv=ps)
    clf_grid.fit(x, y_stacked)
    clf_best = clf_grid.best_estimator_
    scores.append(np.mean(cross_val_score(clf_best, x, y_stacked, cv=ps)))
    hypers.append(clf_best.named_steps['svc'])
    
print('Scores on different feature sets:', scores)
print('Best hyperparameters:', hypers)

In [ ]:
z.shape

## Labeling the unlabeled data

In [ ]:
x_train = x_NoS_train
x_unlabeled = x_NoS_unlabeled
# x_train = pd.concat([x_MP_train, x_NoS_train, x_Ratio_train], axis=1)

clf = Pipeline([('svc', SVC(max_iter=1e6, C=46.416, kernel='rbf', gamma='scale'))])

new_labels = clf.fit(x_train, y_train_b).predict(x_unlabeled)

new_non_fraud_npi = x_unlabeled[new_labels == 'non-fraudulent'].index

len(new_non_fraud_npi)

In [ ]:
## Extracting names corresponding to the fraudulent npis

data2018 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2018_CA_Ophthalmology.csv')
data2017 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017_CA_Ophthalmology.csv')
data2016 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016_CA_Ophthalmology.csv')
data2015 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2015_CA_Ophthalmology.csv')
data2014 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2014_CA_Ophthalmology.csv')
data2013 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_CY2013_CA_Ophthalmology.csv')
data2013 = data2013.rename(columns={'National Provider Identifier ':'National Provider Identifier'})
data2012 = pd.read_csv('D:\\Anaconda\\Jupyter Directory\\Medicare Fraud Detection\\MedicareData_California_Opthalmology\\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_CY2012_CA_Ophthalmology.csv')


In [ ]:
df = pd.DataFrame(index=new_non_fraud_npi, columns=['Last Name', 'First Name'], dtype='str')

for npi in new_non_fraud_npi:
    d = data2012.loc[data2012['National Provider Identifier'] == npi]
    if d.shape[0] != 0:
        df.loc[npi] = d.iloc[0][['Last Name/Organization Name', 'First Name']].values

df.to_csv(r'D:\Anaconda\Jupyter Directory\Medicare Fraud Detection\MedicareData_California_Opthalmology\LoN_Mar22_potentially_nonfraud.csv')

## Label Propagation

In [ ]:
## Labeling the unlabeled data with -1

y_num = y_stacked.copy()
y_num[y_num == 'non-fraudulent'] = 0
y_num[y_num == 'high utilization'] = 1
y_num[y_num == 'fraudulent'] = 2

# flag = pd.Series(data=[-1] * len(data_merged[0].index), index=data_merged[0].index, dtype='int', name='flag')
# for npi in non_fraud:
#      flag.at[npi] = 0
# for npi in high_utilization:
#     flag.at[npi] = 1
# for npi in fraud:
#     flag.at[npi] = 2
    
# x_MP_v, x_NoS_v, x_Ratio_v = x_MP.values, x_NoS.values, x_Ratio.values
# y_v = flag.values

y_lp = y_num.copy()
y_lp[len_train:] = -1
y_lp = y_lp.astype('float64')

In [ ]:
# x_MP_c = [x_MP_fraud, x_MP_high_utilization, x_MP_non_fraud, x_MP_unlabeled] = \
# x_MP.loc[flag == 'fraudulent'], x_MP.loc[flag == 'high utilization'], x_MP.loc[flag == 'non-fraudulent'], x_MP.loc[flag == -1]

# x_NoS_c = [x_NoS_fraud, x_NoS_high_utilization, x_NoS_non_fraud, x_NoS_unlabeled] = \
# x_NoS.loc[flag == 'fraudulent'], x_NoS.loc[flag == 'high utilization'], x_NoS.loc[flag == 'non-fraudulent'], x_NoS.loc[flag == -1]

# x_Ratio_c = [x_Ratio_fraud, x_Ratio_high_utilization, x_Ratio_non_fraud, x_Ratio_unlabeled] = \
# x_Ratio.loc[flag == 'fraudulent'], x_Ratio.loc[flag == 'high utilization'], x_Ratio.loc[flag == 'non-fraudulent'], x_Ratio.loc[flag == -1]

# y_train = flag[flag != -1]

In [ ]:
y_num

In [ ]:
## Implementing KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
x_train = x_stacked
y = y_num[:len_train].astype('float64')

predictions = []
for x in x_train:
    x_labeled, x_unlabeled = x[:len_train], x[len_train:]
    knn.fit(x_labeled, y)
    predictions.append(knn.predict(x_unlabeled))
    
# DF = pd.DataFrame(data=np.array(predictions).T, index=flag[flag==-1].index, columns=['MP', 'NoS', 'Ratio'], dtype='int')
# DF.loc[knn_picked].iloc[10:20]
# s = pd.Series(data=np.sum(np.array(predictions).T, axis=1), index=flag[flag==-1].index, dtype='int')
# knn_picked = s[s>2].index
# DF.loc[knn_picked]

In [ ]:
n = 2
sum(predictions[n] == y_num[len_train:]) / len(predictions[n])

In [ ]:
###### Implementing the label propagation
import sklearn
from sklearn.semi_supervised import LabelPropagation

x = x_stacked[1]

# def rbf_kernel_safe(X, Y=None, gamma=None): 
#     X, Y = sklearn.metrics.pairwise.check_pairwise_arrays(X, Y) 
#     if gamma is None: 
#         gamma = 1.0 / X.shape[1] 
#     K = sklearn.metrics.pairwise.euclidean_distances(X, Y, squared=True) 
#     K *= -gamma 
#     K -= K.max()
#     np.exp(K, K)    # exponentiate K in-place 
#     return K 
  
lp = LabelPropagation(kernel='rbf', gamma=10**-7, tol=10**-6, max_iter=10**6)
lp.fit(x, y_lp)
labels = lp.transduction_

In [ ]:
sum(labels[len_train:] == y_num[len_train:]) / len(labels[len_train:])

In [ ]:
np.sqrt(np.sum((x[0]-x[200]) ** 2))